In [40]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [41]:
#importing libraries 
import sklearn
import numpy as np
from glob import glob
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline 

import string
from os import listdir
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from pickle import dump
from string import punctuation

from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers import LSTM
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from pickle import load

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

import csv, string

In [42]:
categories = ['Gender abusive hate', 'Geopoitical Hate','Religious Hate','Political Normal','Political Hate','Personal Hate']
num_of_labels=len(categories)
categories_dict={key:value for value,key in enumerate(categories)}
categories_inverse_dict={key:value for key,value in enumerate(categories)}
print(categories_dict)

{'Geopoitical Hate': 1, 'Political Hate': 4, 'Political Normal': 3, 'Personal Hate': 5, 'Religious Hate': 2, 'Gender abusive hate': 0}


In [43]:
stop_words='stopwords_bn.txt'
text_data=[]
with open(stop_words,'r',encoding='utf-8') as temp_output_file:
    reader=csv.reader(temp_output_file, delimiter='\n')
    for row in reader:
        text_data.append(row)
stop_word_list=[x[0] for x in text_data]
print(stop_word_list[0:2])
print(type(stop_word_list))

['অতএব', 'অথচ']
<class 'list'>


In [44]:
def save_dataset(dataset, filename):
    dump(dataset, open(filename,'wb'))
    print('Saved :%s' % filename)

def load_doc(filename):
    file=open(filename,'r',encoding='utf-8')
    text=file.read()
    file.close()
    return text

def clean_doc(doc, stop_word_list):
    # split into tokens by white space
    sentences=list()
    
    for sentence in doc:
        # remove punctuation from each token
        
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in sentence.split(' ')]  # I belive word in sentence
        sentences.append(' '.join(tokens))
    return sentences

In [45]:
def load_data_and_labels(filename,stop_word_list):
    doc=load_doc(filename)
    x_doc=list()
    label=list()
    for line in doc.split('\n'):
        x_and_label=line.split('\t')
        #print(x_and_label)
        x_doc.append(x_and_label[1])
        label.append(categories_dict[x_and_label[0]])
        
    #print(x_doc)
    #print(label)
    trainX = clean_doc(x_doc, stopwords)
    return trainX, label
    

In [46]:
X_train, y_train = load_data_and_labels('Hate_Speech_Train.csv',stop_word_list)

In [62]:
print(X_train[0:10])

[[ 137  238 1016 ...    0    0    0]
 [   2  239  138 ...    0    0    0]
 [  35  139  140 ...    0    0    0]
 ...
 [   6   47    2 ...    0    0    0]
 [  36    8  580 ...    0    0    0]
 [ 298   98  138 ...    0    0    0]]


In [48]:
def create_tokenizer(lines):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def encode_text(tokenizer, lines, length):
    #print(lines)
    encoded=tokenizer.texts_to_sequences(lines)
    print(encoded[0:10])
    padded= pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [49]:
train_tokenizer=create_tokenizer(X_train)

In [50]:
trainLength=300
vocab_size=len(train_tokenizer.word_index)+1
print('Max document length: %d' % trainLength)
print('Vocabulary size: %d' % vocab_size)
print()
X_train = encode_text(train_tokenizer, X_train, trainLength)
print(X_train.shape)
#trainX[2]

Max document length: 300
Vocabulary size: 3417

[[137, 238, 1016, 576, 1017, 577, 65], [2, 239, 138, 1018, 32, 46, 32, 1019, 240, 578], [35, 139, 140, 1020, 1, 1021, 4, 1], [13, 81, 1022, 1023, 33, 1024, 14, 4, 1025], [295, 1026, 6, 1027, 162], [141, 114, 1028, 1029, 1030, 1031, 6, 1032, 1033, 579, 1034, 1035], [1036, 296, 1037, 1038, 1039, 1, 1040, 1041, 241, 97], [6, 47, 2, 1042, 1043, 163, 1044, 1045, 389, 297, 241, 1046, 1047, 1048, 24], [36, 8, 580, 1049, 1050, 1051, 1052], [298, 98, 138, 47, 191, 1053, 14, 1054, 1055]]
(877, 300)


In [51]:
word_vectors = Word2Vec.load('C:/Users/admin-karim/Desktop/BengWord2Vec/posts.bin')

In [52]:
NUM_WORDS=20000
EMBEDDING_DIM=300

vocabulary_size=len(train_tokenizer.word_index)+1
word_index=tokenizer.word_index
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector=word_vectors[word]
        embedding_matrix[i]=embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

c:\users\admin-karim\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [53]:
embedding_layer=Embedding(vocabulary_size, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)

In [54]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	input1 = Input(shape=(length,))
	embedding_layer_1 = embedding_layer(input1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding_layer_1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
    
	# channel 2
	input2 = Input(shape=(length,))
	embedding_layer_2 = embedding_layer(input2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding_layer_2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
    
	# channel 3
	input3 = Input(shape=(length,))
	embedding_layer_3 = embedding_layer(input3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding_layer_3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
    
	# merge
	CNN_layer = concatenate([flat1, flat2, flat3])
    
	# LSTM
	x = embedding_layer(input3)
	LSTM_layer = LSTM(128)(x)

	CNN_LSTM_layer = concatenate([LSTM_layer, CNN_layer])
    
	# interpretation
	dense1 = Dense(10, activation='relu')(CNN_LSTM_layer)
	outputs = Dense(num_of_labels, activation='softmax')(dense1)
	model = Model(inputs=[input1, input2, input3], outputs=outputs)
    
	# compile
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
	# summarize
	print(model.summary())
    
	return model

In [55]:
model= define_model(EMBEDDING_DIM,vocabulary_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 300, 300)     1025100     input_4[0][0]                    
                                                                 input_5[0][0]                    
          

In [56]:
from keras.utils import to_categorical

def toCategorical(y):
    y = to_categorical(y)
    return y

In [57]:
y_train = toCategorical(y_train)

In [58]:
model.fit([X_train,X_train,X_train], y_train, epochs=20, batch_size=128, validation_split=0.2)

Train on 701 samples, validate on 176 samples
Epoch 1/20
701/701 [==============================] - 12s 17ms/step - loss: 1.6760 - acc: 0.2539 - val_loss: 3.3705 - val_acc: 0.0398
Epoch 2/20
701/701 [==============================] - 9s 14ms/step - loss: 1.3261 - acc: 0.4579 - val_loss: 5.2184 - val_acc: 0.0000e+00
Epoch 3/20
701/701 [==============================] - 9s 13ms/step - loss: 1.1300 - acc: 0.5449 - val_loss: 6.8787 - val_acc: 0.0000e+00
Epoch 4/20
701/701 [==============================] - 12s 16ms/step - loss: 1.0058 - acc: 0.5877 - val_loss: 8.1012 - val_acc: 0.0000e+00
Epoch 5/20
701/701 [==============================] - 11s 16ms/step - loss: 0.8959 - acc: 0.6391 - val_loss: 9.2797 - val_acc: 0.0000e+00
Epoch 6/20
701/701 [==============================] - 22s 32ms/step - loss: 0.8244 - acc: 0.6534 - val_loss: 9.5983 - val_acc: 0.0000e+00
Epoch 7/20
701/701 [==============================] - 24s 34ms/step - loss: 0.7083 - acc: 0.7361 - val_loss: 9.9701 - val_acc: 0.005

In [59]:
# save the model
model.save('MConv_LSTM_model.h5')

In [60]:
X_test, y_test = load_data_and_labels('Hate_Speech_Test.csv',stop_word_list)
y_test = toCategorical(y_test)

In [61]:
from pickle import load
from numpy import array
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 
    
tokenizer_test = create_tokenizer(X_test)

trainLength=300
vocab_size=len(tokenizer_test.word_index)+1
print('Max document length: %d' % trainLength)
print('Vocabulary size: %d' % vocab_size)
print()
X_test = encode_text(tokenizer_test, X_test, trainLength)
print(X_test.shape)
#trainX[2]

# load the model
model = load_model('MConv_LSTM_model.h5') 

# evaluate model on test dataset dataset
loss, acc = model.evaluate([X_test,X_test,X_test], y_test, verbose=0)
print('Test accuracy: %f' % (acc*100))

Max document length: 300
Vocabulary size: 1616

[[222, 409, 410, 411, 412, 22], [145, 47, 81, 48, 146, 413, 414, 82, 109], [415, 223, 42, 224, 5, 416], [417, 418, 419, 225, 23], [110, 10, 420, 226, 227, 421, 228, 422, 423, 424, 229, 230, 147, 231, 1, 36, 425, 37, 426], [148, 427, 38, 428, 232, 83, 111, 38, 429, 1], [26, 233, 430, 149], [431, 432, 14, 433, 434], [435, 23, 234, 235, 236, 436], [26, 17, 47]]
(323, 300)
Test accuracy: 30.650155
